In [1]:
import tensorflow as tf
import transformers
import data
from keras import backend as K
import pandas as pd
import re
from nltk.corpus import  stopwords

2024-12-02 01:41:21.228384: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /lib:/lib/
2024-12-02 01:41:21.228401: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# model_name = "FacebookAI/roberta-base"
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = transformers.RobertaTokenizer.from_pretrained(model_name)

In [3]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [4]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = text.split()
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_text)

In [5]:
twitter_train = pd.read_csv('data/Mutant Datasets/dataset_m_111.csv')
twitter_train['tweet'] = twitter_train['tweet'].apply(remove_stopwords)
twitter_train['tweet'] = twitter_train['tweet'].apply(remove_urls)
twitter_train['text'] = twitter_train['tweet']
twitter_train['source'] = "twitter"
twitter_train = twitter_train[['text', 'sarcastic', 'source']]

In [6]:
reddit_train = pd.read_csv('data/soraby_sarcasm2/sarc_train.csv')
reddit_train['text'] = reddit_train['text'].apply(remove_stopwords)
reddit_train['text'] = reddit_train['text'].apply(remove_urls)
reddit_train['sarcastic'] = reddit_train['class']
reddit_train['source'] = "reddit"
reddit_train = reddit_train[['text', 'sarcastic', 'source']]

In [13]:
df = pd.concat([reddit_train, twitter_train],axis=0, ignore_index=True)

In [14]:
from sklearn.utils import resample

minority_class = df[df['sarcastic'] == 1]
majority_class = df[df['sarcastic'] == 0]
minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)

In [15]:
df = pd.concat([majority_class, minority_upsampled])

In [10]:
df.to_csv("data/mydata/twitter_reddit_merge.csv")

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['sarcastic'], stratify=df['sarcastic'], test_size=0.2, random_state=42)

In [16]:
def tokenize_function(texts):
    print(type(texts))
    encodings = tokenizer(
        texts, 
        max_length=128, 
        padding="max_length", 
        truncation=True, 
        return_tensors="tf"
    )
    return encodings

In [17]:
train_enc = tokenize_function(x_train.values.tolist())
test_enc = tokenize_function(x_test.values.tolist())

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_enc), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_enc), y_test))

batch_size = 16
train_dataset = train_dataset.shuffle(len(y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(len(y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

<class 'list'>
<class 'list'>


In [14]:
config = transformers.BertConfig.from_pretrained(model_name,
                                    output_hidden_states=True)
roberta_model = transformers.TFRobertaModel.from_pretrained(model_name, config=config)
input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
roberta_outputs = roberta_model(input_ids=input_ids, attention_mask=attention_mask)

x1 = tf.keras.layers.Dropout(0.1)(roberta_outputs.last_hidden_state)
x1 = tf.keras.layers.Conv1D(filters=128, kernel_size=3,padding='same')(x1)
x1 = tf.keras.layers.LeakyReLU()(x1)
x1 = tf.keras.layers.GlobalMaxPooling1D()(x1)
output = tf.keras.layers.Dense(1, activation="softmax")(x1)
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
print(model.summary())

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel fo

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [15]:
def f1_score(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # Threshold predictions
    tp = K.sum(y_true * y_pred)  # True positives
    fp = K.sum((1 - y_true) * y_pred)  # False positives
    fn = K.sum(y_true * (1 - y_pred))  # False negatives

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-8),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(), 
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.BinaryCrossentropy(),
        f1_score
    ]
)


# %%
# model.fit(dataset, epochs=5)
out = model.fit(train_dataset, epochs=1)
results = model.evaluate(test_dataset, batch_size=128)
print(out.history)

import json
with open("results/roberta_cnn.json", "w+") as file:
    file.write(json.dumps(out.history))

model.save("output/sarc_detect_2.keras")

 51/524 [=>............................] - ETA: 11:49 - loss: 0.9141 - binary_accuracy: 0.4730 - false_negatives: 0.0000e+00 - false_positives: 430.0000 - binary_crossentropy: 0.9141 - f1_score: 0.6301

KeyboardInterrupt: 